# 1) Extract beta event information from the entire sample

Using the spectral events toolbox, derive at the number, duration, peak power, and frequency span of beta events across the timepoints for all series. Further below I'll actually drill down into seeing what's going on with individual timepoints or making comparisons/predictions.

In [ ]:
# load in packages
%matplotlib widget

import sys
import os.path as op
import os
from glob import glob
import pickle
import statistics

import numpy as np
import scipy.stats as stats
from scipy.io import loadmat
import matplotlib.pyplot as plt
import seaborn as sns

from mne.io import read_epochs_eeglab
import mne

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
import pandas as pd

# set path to SpectralEvents if necessary
sys.path.append('/gpfs/home/ddiesbur/brainstorm-ws/spectral-events')
import spectralevents as se

In [ ]:
# Load in the data
t1_fname = []
t2_fname = []
t1_val_fname = []
t2_val_fname = []
# iterating across and within series to get lists of our files.
# later on we'll check the subject ID in file names and series number
# to make sure we're comparing times for the same subject within the relevant series
# NOTE THAT BEFORE I RAN THIS NOTEBOOK I USED THE MATLAB FILE applyCSD.m TO ADD A PREPROCESSING STEP
# IN WHICH I APPLIED A CURRENT SOURCE DENSITY TRANSFORM TO MINIMIZE VOLUME CONDUCTION 
# training sets
data_dir = '/gpfs/home/ddiesbur/scratch/CSD/'
# get T1s
for file_idx in os.listdir(data_dir):
    if ("Pre_Session1" in file_idx) & (".set" in file_idx):
        t1_fname.append(op.join(data_dir,file_idx))
# get T2s    
for file_idx in os.listdir(data_dir):
    if ("Post_Session1" in file_idx) & (".set" in file_idx):
        t2_fname.append(op.join(data_dir,file_idx))
        
# testing/validation sets        
data_val_dir = '/gpfs/home/ddiesbur/scratch/CSD_val/'
# get T1s
for file_idx in os.listdir(data_val_dir):
    if ("Pre_Session1" in file_idx) & (".set" in file_idx):
        t1_val_fname.append(op.join(data_val_dir,file_idx))
# get T2s    
for file_idx in os.listdir(data_val_dir):
    if ("Post_Session1" in file_idx) & (".set" in file_idx):
        t2_val_fname.append(op.join(data_val_dir,file_idx))

Extract beta evs for T1
(Note that there are commented-out lines to toggle for whether I'm getting evs from the training or the validation sets.)

In [ ]:
# settings for extraction
n_times = 2000
samp_freq = 500
freqs = list(range(2, 45 + 1))  # fequency values (Hz) over which to calculate TFR
times = np.arange(n_times) / samp_freq  # seconds
event_band = [15, 29]  # beta band (Hz)
thresh_FOM = 6.0  # factor-of-the-median threshold

In [ ]:
# Get evs from T1 sets
# For the sake of saving memory and not overwhelming my data directory, I'll be running each dataset through TFR 
# conversion and spectral event detection right away, deleting the full data when I'm done with it.
# Spectral event info is saved in scratch with pickle.

#t1_save_dir = '/gpfs/home/ddiesbur/scratch/spectral_events/T1/'
t1_save_dir = '/gpfs/home/ddiesbur/scratch/spectral_events/T1_val/'

for subj_idx, subj_file in enumerate(t1_val_fname):
#for subj_idx, subj_file in enumerate(t1_fname):
    # load eeg data
    subj_ID = subj_file[-46:-42] # subject ID, they're not in order in the list, but we'll
    # just grab this and the series ID so the new filename has that info for later
    series_ID = subj_file[-22:-21]
    epochs_t1 = read_epochs_eeglab(subj_file)
    spec_events_dict = {}

    # iterate channels
    for chan_idx,chan_name in enumerate(epochs_t1.ch_names):

        data_t1 = epochs_t1.get_data(chan_idx).squeeze()  # epoch x time for the channel of loop iteration

        # TFR conversion
        tfrs_t1 = se.tfr(data_t1, freqs, samp_freq)

        # Get spectral events
        spec_events = se.find_events(tfr=tfrs_t1, times=times, freqs=freqs,
                                     event_band=event_band, threshold_FOM=thresh_FOM)
        # put in dictionary with all other chans, so that evs can be indexed with chan name
        spec_events_dict[chan_name] = spec_events

        del spec_events, data_t1, tfrs_t1
        
    # make filename and pickle
    save_file = op.join(t1_save_dir,'Betaevs_'+subj_ID+'_S'+series_ID+'_T1.pkl')
    with open(save_file, 'wb') as file:
        # A new file will be created
        pickle.dump(spec_events_dict, file)

del epochs_t1 # save memory

Extract beta evs for T2

In [ ]:
# get evs from T2 sets
#t2_save_dir = '/gpfs/home/ddiesbur/scratch/spectral_events/T2/'
t2_save_dir = '/gpfs/home/ddiesbur/scratch/spectral_events/T2_val/'

for subj_idx, subj_file in enumerate(t2_val_fname):
#for subj_idx, subj_file in enumerate(t2_fname):
    # load eeg data
    subj_ID = subj_file[-47:-43] # subject ID, they're not in order in the list
    series_ID = subj_file[-22:-21]
    epochs_t2 = read_epochs_eeglab(subj_file)
    spec_events_dict = {}

    # iterate channels
    for chan_idx,chan_name in enumerate(epochs_t2.ch_names):

        data_t2 = epochs_t2.get_data(chan_idx).squeeze()  # epoch x time for the channel of loop iteration

        # TFR conversion
        tfrs_t2 = se.tfr(data_t2, freqs, samp_freq)

        # Get spectral events
        spec_events = se.find_events(tfr=tfrs_t2, times=times, freqs=freqs,
                                     event_band=event_band, threshold_FOM=thresh_FOM)
        # put in dictionary with all other chans
        spec_events_dict[chan_name] = spec_events

        del spec_events, data_t2, tfrs_t2
        
    # make filename and pickle
    save_file = op.join(t2_save_dir,'Betaevs_'+subj_ID+'_S'+series_ID+'_T2.pkl')
    with open(save_file, 'wb') as file:
        # A new file will be created
        pickle.dump(spec_events_dict, file)

del epochs_t2

# 2) Get information about beta events, visualize, and predict

In [ ]:
# Ok, now that we've made new pickle files that hold our dictionaries of beta ev
# characteristics, set up lists of those files for each time condition, which
# we'll iterate through to extract important info about the beta evs.
evsets_t1 = []
evsets_t2 = []
evsets_val_t1 = []
evsets_val_t2 = []

evs_t1_dir = '/gpfs/home/ddiesbur/scratch/spectral_events/T1/'
evs_t2_dir = '/gpfs/home/ddiesbur/scratch/spectral_events/T2/'
evs_t1_val_dir = '/gpfs/home/ddiesbur/scratch/spectral_events/T1_val/'
evs_t2_val_dir = '/gpfs/home/ddiesbur/scratch/spectral_events/T2_val/'

# training sets
for file_idx in os.listdir(evs_t1_dir):
    if file_idx.endswith(".pkl"):
        evsets_t1.append(op.join(evs_t1_dir,file_idx))
for file_idx in os.listdir(evs_t2_dir):
    if file_idx.endswith(".pkl"):
        evsets_t2.append(op.join(evs_t2_dir,file_idx))
        
# validation sets
for file_idx in os.listdir(evs_t1_val_dir):
    if file_idx.endswith(".pkl"):
        evsets_val_t1.append(op.join(evs_t1_val_dir,file_idx))
for file_idx in os.listdir(evs_t2_val_dir):
    if file_idx.endswith(".pkl"):
        evsets_val_t2.append(op.join(evs_t2_val_dir,file_idx))

In [ ]:
# necessary functions - each takes a dictionary of beta ev characteristics from one channel's worth of
# data as input.
def avg_event_count(subj_spec_events):

    # initialize the event count to 0
    event_count = []

    # iterate over trials and add event counts depending on trial type
    for trial_idx, trial_events in enumerate(subj_spec_events):
        event_count.append(len(trial_events)) # iterating over trials (epochs), make 
        # list of how many beta evs in the trial (per 4s)
        
    return statistics.mean(event_count) # take avg, which gives us rate of beta evs per 4s epoch

def avg_event_peakfreq(subj_spec_events):

    # initialize the event count to 0
    event_freq = []

    # iterate over trials and add event counts depending on trial type
    for trial_idx in enumerate(subj_spec_events):
        for event_idx in enumerate(subj_spec_events[int(trial_idx[0])]):
            event_freq.append(subj_spec_events[int(trial_idx[0])][int(event_idx[0])]['Peak Frequency'])
            # iterating over trials and beta evs, grab peak frequency of each ev observed
            
    return statistics.mean(event_freq) # take avg, which gives us avg peak freq

def avg_event_dur(subj_spec_events):

    # initialize the event count to 0
    event_dur = []

    # iterate over trials and add event counts depending on trial type
    for trial_idx in enumerate(subj_spec_events):
        for event_idx in enumerate(subj_spec_events[int(trial_idx[0])]):
            event_dur.append(subj_spec_events[int(trial_idx[0])][int(event_idx[0])]['Event Duration'])
            # iterating over trials and beta evs, grab duration of each ev observed

    return statistics.mean(event_dur) # take avg, which gives us avg ev duration

def avg_event_pow(subj_spec_events):

    # initialize the event count to 0
    event_pow = []

    # iterate over trials and add event counts depending on trial type
    for trial_idx in enumerate(subj_spec_events):
        for event_idx in enumerate(subj_spec_events[int(trial_idx[0])]):
            event_pow.append(subj_spec_events[int(trial_idx[0])][int(event_idx[0])]['Normalized Peak Power'])
            # iterating over trials and beta evs, grab normed power of each ev observed

    return statistics.mean(event_pow) # take avg, which gives us avg ev power

Get information about differences between responders, non responders at baseline (T1) in the training set

In [ ]:
# grab the clinical info we're going to need for our entire dataset
import csv
fields = []
rows = []
filename = '/gpfs/data/brainstorm-ws/data/TRAINING/TRAINING_Demographics and Clinical Outcomes_All Series.csv'
row_subs = []

with open(filename, 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        rows.append(row)
        
# for ease of later search pad sub nums with zeros
for isub in range(len(rows)):
    row_subs.append(rows[isub][0].zfill(4))

In [ ]:
# preallocate
ev_count_t1 = np.empty((0,62))
ev_freq_t1 = np.empty((0,62))
ev_dur_t1 = np.empty((0,62))
ev_pow_t1 = np.empty((0,62))
resp = []
noresp = []
resp_or_no = []
field2get = fields.index('LastRespIDSSR')
epochs = read_epochs_eeglab(t1_fname[0]) # grab chan info from one of the eeg sets
pos = epochs.info

# cycle through subjects
for subj_idx, subj_file in enumerate(evsets_t1):
    # load evs data
    subj_ID = subj_file[-14:-10] # subject ID, they're not in order in the list
    series_ID = subj_file[-8:-7]
    
    # mark whether this sub a responder or no
    this_sub = row_subs.index(subj_ID)
    if int(rows[this_sub][field2get])==1:
        resp.append(True)
        noresp.append(False)
    elif int(rows[this_sub][field2get])==0:
        resp.append(False)
        noresp.append(True)
    resp_or_no.append(int(rows[this_sub][field2get]))
    t1_evs_dict = []
    
    # load pickles
    load_file = open(subj_file,'rb')
    t1_evs_dict = pickle.load(load_file)
 
    # peak freq, num, duration, amplitude
    # make empty for each new subject
    ev_freq_chans = []
    ev_count_chans = []
    ev_dur_chans = []
    ev_pow_chans = []

    for chan_name in t1_evs_dict.keys():
        # iterate through channels to get avg beta characteristics at each (avg'ed over all 4s epochs)
        ev_freq_chans.append(avg_event_peakfreq(t1_evs_dict[chan_name]))
        ev_count_chans.append(avg_event_count(t1_evs_dict[chan_name]))
        ev_dur_chans.append(avg_event_dur(t1_evs_dict[chan_name]))
        ev_pow_chans.append(avg_event_pow(t1_evs_dict[chan_name]))
    
    # append avg for each subject as we iterate through them
    ev_freq_t1 = np.append(ev_freq_t1, [ev_freq_chans], axis=0)
    ev_count_t1 = np.append(ev_count_t1, [ev_count_chans], axis=0)
    ev_dur_t1 = np.append(ev_dur_t1, [ev_dur_chans], axis=0)
    ev_pow_t1 = np.append(ev_pow_t1, [ev_pow_chans], axis=0)
    

In [ ]:
# plot results - freq
plt.close('all')
freq_data = np.mean(ev_freq_t1[resp],axis=0)-np.mean(ev_freq_t1[noresp],axis=0)
fig1 = mne.viz.plot_topomap(freq_data, pos,names=list(t1_evs_dict.keys()),show_names=True)

# stats
tvalue, pvalue = stats.ttest_ind(np.mean(ev_freq_t1[resp],axis=1),np.mean(ev_freq_t1[noresp],axis=1))
print('Overall differences in global avg beta ev freq: = '+str(tvalue)+', p = '+str(pvalue))

# t-tests w/ FDR
T = []
P = []
for chan_idx in range(len(t1_evs_dict.keys())):
    tvalue, pvalue = stats.ttest_ind(ev_freq_t1[resp,chan_idx],ev_freq_t1[noresp,chan_idx])
    T.append(tvalue)
    P.append(pvalue)
_,corrP = mne.stats.fdr_correction(P, alpha=0.05, method='indep')    

sig_channels = [i for i in range(len(P)) if P[i] <= 0.05]
print('There are significant differences in beta ev frequency between responders and non-responders at channels '+str(sig_channels)+'.')

In [ ]:
# plot results - counts
plt.close('all')
count_data = np.mean(ev_count_t1[resp],axis=0)-np.mean(ev_count_t1[noresp],axis=0)
fig2 = mne.viz.plot_topomap(count_data, pos,names=list(t1_evs_dict.keys()),show_names=True)

# stats
tvalue, pvalue = stats.ttest_ind(np.mean(ev_count_t1[resp],axis=1),np.mean(ev_count_t1[noresp],axis=1))
print('Overall diff in global evg beta ev count: F = '+str(tvalue)+', p = '+str(pvalue))

# t-tests w/ FDR
T = []
P = []
for chan_idx in range(len(t1_evs_dict.keys())):
    tvalue, pvalue = stats.ttest_ind(ev_count_t1[resp,chan_idx],ev_count_t1[noresp,chan_idx])
    T.append(tvalue)
    P.append(pvalue)
_,corrP = mne.stats.fdr_correction(P, alpha=0.05, method='indep')    
sig_channels = [i for i in range(len(P)) if P[i] <= 0.05]
print('There are significant differences in beta ev count between responders and non-responders at channels '+str(sig_channels)+'.')

In [ ]:
# plot results - duration
plt.close('all')
dur_data = np.mean(ev_dur_t1[resp],axis=0)-np.mean(ev_dur_t1[noresp],axis=0)
fig3 = mne.viz.plot_topomap(dur_data, pos,names=list(t1_evs_dict.keys()),show_names=True);

# stats 
ftvalue, pvalue = stats.ttest_ind(np.mean(ev_dur_t1[resp],axis=1),np.mean(ev_dur_t1[noresp],axis=1))
print('Overall diff in global avg eta ev duration: T = '+str(tvalue)+', p = '+str(pvalue))

# t-tests w/ FDR
T = []
P = []
for chan_idx in range(len(t1_evs_dict.keys())):
    tvalue, pvalue = stats.ttest_ind(ev_dur_t1[resp,chan_idx],ev_dur_t1[noresp,chan_idx])
    T.append(tvalue)
    P.append(pvalue)
_,corrP = mne.stats.fdr_correction(P, alpha=0.05, method='indep')    
sig_channels = [i for i in range(len(P)) if P[i] <= 0.05]
print('There are significant differences in beta ev duration between responders and non-responders at channels '+str(sig_channels)+'.')

In [ ]:
# plot results - power
plt.close('all')
pow_data = np.mean(ev_pow_t1[resp],axis=0)-np.mean(ev_pow_t1[noresp],axis=0)
fig4,ax4 = mne.viz.plot_topomap(pow_data, pos,names=list(t1_evs_dict.keys()),show_names=True);

# stats 
tvalue, pvalue = stats.ttest_ind(np.mean(ev_pow_t1[resp],axis=1),np.mean(ev_pow_t1[noresp],axis=1))
print('Overall diff in global avg beta ev power: T = '+str(tvalue)+', p = '+str(pvalue))

# t-tests w/ FDR
T = []
P = []
for chan_idx in range(len(t1_evs_dict.keys())):
    tvalue, pvalue = stats.ttest_ind(ev_pow_t1[resp,chan_idx],ev_pow_t1[noresp,chan_idx])
    T.append(tvalue)
    P.append(pvalue)
_,corrP = mne.stats.fdr_correction(P, alpha=0.05, method='indep')
sig_channels = [i for i in range(len(P)) if P[i] <= 0.05]
print('There are significant differences in beta ev power between responders and non-responders at channels '+str(sig_channels)+'.')

In [ ]:
chan_idx = list(t1_evs_dict.keys()).index('FC3')
data = [ev_count_t1[resp,chan_idx], ev_count_t1[noresp,chan_idx]]
fig1, ax1 = plt.subplots()
ax1.set_title('Channel-wise diff for resp/non-resp')
ax1.boxplot(data);

Get information about beta ev differences from T1 to T2 at all channels

In [ ]:
# preallocate
ev_count_t1t2 = np.empty((0,62))
ev_freq_t1t2 = np.empty((0,62))
ev_dur_t1t2 = np.empty((0,62))
ev_pow_t1t2 = np.empty((0,62))
resp = []
noresp = []
resp_or_no = []
field2get = fields.index('LastRespIDSSR')
epochs = read_epochs_eeglab(t1_fname[0])
pos = epochs.info

# cycle through subjects
for subj_idx, subj_file in enumerate(evsets_t2):
    # load evs data
    subj_ID = subj_file[-14:-10] # subject ID, they're not in order in the list
    series_ID = subj_file[-8:-7]
    
    # mark whether this sub a responder or no
    this_sub = row_subs.index(subj_ID)
    if int(rows[this_sub][field2get])==1:
        resp.append(True)
        noresp.append(False)
    elif int(rows[this_sub][field2get])==0:
        resp.append(False)
        noresp.append(True)
    resp_or_no.append(int(rows[this_sub][field2get]))
    
    t2_evs_dict = []
    t1_evs_dict = []
    
    # load pickles
    load_file = open(subj_file,'rb')
    t2_evs_dict = pickle.load(load_file)
    # find which sub coresponds to t2
    index = [idx for idx, s in enumerate(evsets_t1) if str(subj_ID+'_S'+series_ID) in s]
    load_file = open(evsets_t1[index[0]],'rb')
    t1_evs_dict = pickle.load(load_file)
    
    # peak freq, num, duration, amplitude
    ev_freq_chans = []
    ev_count_chans = []
    ev_dur_chans = []
    ev_pow_chans = []

    for chan_name in t1_evs_dict.keys():
        ev_freq_chans.append(100*(avg_event_peakfreq(t2_evs_dict[chan_name])-avg_event_peakfreq(t1_evs_dict[chan_name]))/avg_event_peakfreq(t1_evs_dict[chan_name]))
        ev_count_chans.append(100*(avg_event_count(t2_evs_dict[chan_name])-avg_event_count(t1_evs_dict[chan_name]))/avg_event_count(t1_evs_dict[chan_name]))
        ev_dur_chans.append(100*(avg_event_dur(t2_evs_dict[chan_name])-avg_event_dur(t1_evs_dict[chan_name]))/avg_event_dur(t1_evs_dict[chan_name]))
        ev_pow_chans.append(100*(avg_event_pow(t2_evs_dict[chan_name])-avg_event_pow(t1_evs_dict[chan_name]))/avg_event_pow(t1_evs_dict[chan_name]))
        
    ev_freq_t1t2 = np.append(ev_freq_t1t2, [ev_freq_chans], axis=0)
    ev_count_t1t2 = np.append(ev_count_t1t2, [ev_count_chans], axis=0)
    ev_dur_t1t2 = np.append(ev_dur_t1t2, [ev_dur_chans], axis=0)
    ev_pow_t1t2 = np.append(ev_pow_t1t2, [ev_pow_chans], axis=0)
    

In [ ]:
# plot results - freq
plt.close('all')
freq_data = np.mean(ev_freq_t1t2[resp],axis=0)-np.mean(ev_freq_t1t2[noresp],axis=0)
fig1 = mne.viz.plot_topomap(freq_data, pos,names=list(t1_evs_dict.keys()),show_names=True)

# stats
tvalue, pvalue = stats.ttest_ind(np.mean(ev_freq_t1t2[resp],axis=1),np.mean(ev_freq_t1t2[noresp],axis=1))
print('Overall diff in global avg beta ev freq T2-T1: T = '+str(tvalue)+', p = '+str(pvalue))

# t-tests w/ FDR
T = []
P = []
for chan_idx in range(len(t1_evs_dict.keys())):
    tvalue, pvalue = stats.ttest_ind(ev_freq_t1t2[resp,chan_idx],ev_freq_t1t2[noresp,chan_idx])
    T.append(tvalue)
    P.append(pvalue)
_,corrP = mne.stats.fdr_correction(P, alpha=0.05, method='indep')    

sig_channels = [i for i in range(len(P)) if P[i] <= 0.05]
print('There are significant differences in beta ev frequency diff from T2-T1 between responders and non-responders at channels '+str(sig_channels)+'.')

In [ ]:
# plot results - counts
plt.close('all')
count_data = np.mean(ev_count_t1t2[resp],axis=0)-np.mean(ev_count_t1t2[noresp],axis=0)
fig2 = mne.viz.plot_topomap(count_data, pos,names=list(t1_evs_dict.keys()),show_names=True)

# stats
tvalue, pvalue = stats.ttest_ind(np.mean(ev_count_t1t2[resp],axis=1),np.mean(ev_count_t1t2[noresp],axis=1))
print('Overall diff in global avg beta ev count T2-T1: T = '+str(tvalue)+', p = '+str(pvalue))

# t-tests w/ FDR
T = []
P = []
for chan_idx in range(len(t1_evs_dict.keys())):
    tvalue, pvalue = stats.ttest_ind(ev_count_t1t2[resp,chan_idx],ev_count_t1t2[noresp,chan_idx])
    T.append(tvalue)
    P.append(pvalue)
_,corrP = mne.stats.fdr_correction(P, alpha=0.05, method='indep')    
sig_channels = [i for i in range(len(P)) if P[i] <= 0.05]
print('There are significant differences in beta ev count from T2-T1 between responders and non-responders at channels '+str(sig_channels)+'.')

In [ ]:
# plot results - duration
plt.close('all')
dur_data = np.mean(ev_dur_t1t2[resp],axis=0)-np.mean(ev_dur_t1t2[noresp],axis=0)
fig3 = mne.viz.plot_topomap(dur_data, pos,names=list(t1_evs_dict.keys()),show_names=True);

# stats 
tvalue, pvalue = stats.ttest_ind(np.mean(ev_dur_t1t2[resp],axis=1),np.mean(ev_dur_t1t2[noresp],axis=1))
print('Overall diff in global avg beta ev duration T2-T1: T = '+str(tvalue)+', p = '+str(pvalue))

# t-tests w/ FDR
T = []
P = []
for chan_idx in range(len(t1_evs_dict.keys())):
    tvalue, pvalue = stats.ttest_ind(ev_dur_t1t2[resp,chan_idx],ev_dur_t1t2[noresp,chan_idx])
    T.append(tvalue)
    P.append(pvalue)
_,corrP = mne.stats.fdr_correction(P, alpha=0.05, method='indep')    
sig_channels = [i for i in range(len(P)) if P[i] <= 0.05]
print('There are significant differences in beta ev duration from T2-T1 between responders and non-responders at channels '+str(sig_channels)+'.')

In [ ]:
# plot results - power
plt.close('all')
pow_data = np.mean(ev_pow_t1t2[resp],axis=0)-np.mean(ev_pow_t1t2[noresp],axis=0)
fig4,ax4 = mne.viz.plot_topomap(pow_data, pos,names=list(t1_evs_dict.keys()),show_names=True);

# stats 
tvalue, pvalue = stats.ttest_ind(np.mean(ev_pow_t1t2[resp],axis=1),np.mean(ev_pow_t1t2[noresp],axis=1))
print('Overall diff in global avg beta ev power T2-T1: T = '+str(tvalue)+', p = '+str(pvalue))

# t-tests w/ FDR
T = []
P = []
for chan_idx in range(len(t1_evs_dict.keys())):
    tvalue, pvalue = stats.ttest_ind(ev_pow_t1t2[resp,chan_idx],ev_pow_t1t2[noresp,chan_idx])
    T.append(tvalue)
    P.append(pvalue)
_,corrP = mne.stats.fdr_correction(P, alpha=0.05, method='indep')
sig_channels = [i for i in range(len(P)) if P[i] <= 0.05]
print(P)
print('There are significant differences in beta ev power from T2-T1 between responders and non-responders at channels '+str(sig_channels)+'.')

In [ ]:
chan_idx = list(t1_evs_dict.keys()).index('C2')
data = [ev_dur_t1t2[resp,chan_idx], ev_dur_t1t2[noresp,chan_idx]]
fig1, ax1 = plt.subplots()
ax1.set_title('Channel-wise diff for resp/non-resp')
ax1.boxplot(data);

Predict responders/nonresponders by event characteristics

In [ ]:
# THIS CELL DOES A GRID SEARCH
chan2try = 'C2'
del optimal_params,logRegOutput
X = ev_dur_t1t2[:,list(t1_evs_dict.keys()).index(chan2try)]
X = np.reshape(X,(-1, 1))
y = np.array(resp_or_no)
# split into training/testing subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=16)

# possible settings to try with a grid search over log reg settings
param_grid = [
    {'penalty' : ['none', 'l1', 'l2', 'elasticnet'],
     'C' : np.logspace(-4, 4, 20),
     'solver' : ['lbfgs', 'newton-cg', 'liblinear', 'sag', 'saga'],
     'max_iter' : [100, 1000, 2500, 5000, 10000]}]

# perform grid search
optimal_params = GridSearchCV(
    LogisticRegression(), 
    param_grid, 
    verbose = False,
    n_jobs = -1)

# fit the model with ideal params
optimal_params.fit(X_train, y_train);
print('Our optimized parameters:')
print(optimal_params.best_params_)
logRegOutput = LogisticRegression(C=optimal_params.best_params_['C'], 
                            penalty=optimal_params.best_params_['penalty'],
                            solver = optimal_params.best_params_['solver'],
                            max_iter = optimal_params.best_params_['max_iter'],
                            random_state = 16)
logRegOutput.fit(X_train, y_train)

# use the model to predict outcomes in held-out testing subset
y_pred = logRegOutput.predict(X_test)

In [ ]:
# rough accuracy score
predictionScore = logRegOutput.score(X_test,y_test)
print('Prediction: ' + str(round(predictionScore*100)) + "%")

In [ ]:
# make confusion matrix to see how off we were for responders, nonresponders
cnf_matrix = metrics.confusion_matrix(y_test,y_pred)

class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

#Text(0.5,257.44,'Predicted label');

# get precision, recall, F1
target_names = ['non-responder','responder']
print(classification_report(y_test, y_pred, target_names=target_names,zero_division=0))

In [ ]:
# AUC score
plt.close('all')
y_pred_proba = logRegOutput.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)

plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

Get beta ev T1-T2 info for the validation sets to make predictions

In [ ]:
# preallocate
ev_dur_val_t1t2 = np.empty((0,62))
epochs = read_epochs_eeglab(t1_val_fname[0])
pos = epochs.info

# cycle through subjects
for subj_idx, subj_file in enumerate(evsets_val_t2):
    # load evs data
    subj_ID = subj_file[-14:-10] # subject ID, they're not in order in the list
    series_ID = subj_file[-8:-7]
    
    t2_evs_dict = []
    t1_evs_dict = []
    
    # load pickles
    load_file = open(subj_file,'rb')
    t2_evs_dict = pickle.load(load_file)
    # find which sub coresponds to t2
    index = [idx for idx, s in enumerate(evsets_val_t1) if str(subj_ID+'_S'+series_ID) in s]
    load_file = open(evsets_val_t1[index[0]],'rb')
    t1_evs_dict = pickle.load(load_file)
    
    # ev duration
    ev_dur_chans = []

    for chan_name in t1_evs_dict.keys():
        ev_dur_chans.append(100*(avg_event_dur(t2_evs_dict[chan_name])-avg_event_dur(t1_evs_dict[chan_name]))/avg_event_dur(t1_evs_dict[chan_name]))
        
    ev_dur_val_t1t2 = np.append(ev_dur_val_t1t2, [ev_dur_chans], axis=0)
    

In [ ]:
# plot results - duration
plt.close('all')
dur_data = np.mean(ev_dur_val_t1t2,axis=0)
fig3 = mne.viz.plot_topomap(dur_data, pos,names=list(t1_evs_dict.keys()),show_names=True);

In [ ]:
# use the model we defined previously to predict outcomes in the entire validation set
X_validation = ev_dur_val_t1t2[:,list(t1_evs_dict.keys()).index('C2')]
X_validation = np.reshape(X_validation,(-1, 1))
y_pred_val = logRegOutput.predict(X_validation)

In [ ]:
print(y_pred_val,np.mean(y_pred_val))

In [ ]:
# cycle through subjects
for subj_idx, subj_file in enumerate(evsets_val_t1):
    # load evs data
    subj_ID = subj_file[-14:-10] 
    series_ID = subj_file[-8:-7]
    subsin1[subj_idx] = subj_ID
    seriesin1[subj_idx] = series_ID
    
    index = [idx for idx, s in enumerate(evsets_val_t2) if str(subj_ID+'_S'+series_ID) in s]
    
    if not index:
        print('Subject '+str(subj_ID)+' series '+str(series_ID)+' does not have a T2 recording.')

In [ ]:
import csv

# make and save .csv file with subject numbers and predictions
with open('validation_predictions.csv','w',newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    for subj_idx, subj_file in enumerate(evsets_val_t2):
        # load evs data
        subj_ID = subj_file[-14:-10] # subject ID, they're not in order in the list
        series_ID = subj_file[-8:-7]    

        nextrow = [str(subj_ID.lstrip('0')),str(series_ID),str(y_pred_val[subj_idx])]
        # add ID and prediction value
        csv_writer.writerow(nextrow)